In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- 1. O CENÁRIO ---
# Imagine uma linha de pixels.
# Queremos colar uma "montanha" (source) em uma "planície" (target).

N = 10 # Número de pontos desconhecidos (tamanho da nossa colagem)

# O "Target" (Fundo): Uma planície na altura 10
# Nosso objetivo é que as bordas da colagem se conectem suavemente a este 10.
altura_esquerda = 10
altura_direita = 10

# O "Source" (Objeto a ser colado): Uma montanha
# No Poisson Editing, NÃO importam os valores absolutos (ex: 50, 60),
# importam as MUDANÇAS (Gradientes).
source_pattern = np.array([0, 2, 4, 6, 8, 8, 6, 4, 2, 0])

# --- 2. A MATEMÁTICA (Construindo Ax = b) ---

# A Matriz A (Laplaciano 1D)
# Representa a segunda derivada discreta: f(x+1) + f(x-1) - 2f(x)
# Em formato matricial, a diagonal é -2 e os vizinhos são 1.
# (Nota: Em 2D a diagonal é 4 e vizinhos -1, o sinal inverte mas a lógica é a mesma)
A = np.zeros((N, N))
for i in range(N):
    A[i, i] = -2       # O ponto central
    if i > 0:
        A[i, i-1] = 1  # Vizinho da esquerda
    if i < N - 1:
        A[i, i+1] = 1  # Vizinho da direita

print("Matriz A (Operador Laplaciano):")
print(A)

# O Vetor b (Guidance Field + Bordas)
# b deve conter o Laplaciano do Source (a "curvatura" da montanha)
# MENOS as condições de contorno (Dirichlet)

# 1. Calcular o Laplaciano do Source (o que queremos copiar)
# (Vamos usar a mesma lógica: vizinho esq + vizinho dir - 2*centro)
# Para simplificar, vamos usar np.diff duas vezes, que é a derivada segunda
laplaciano_source = np.zeros(N)
# Simulação manual da derivada segunda no source_pattern para os pontos internos
# (Isso define a FORMA da montanha)
# Nota: na prática real faremos isso com convolução
pad_source = np.pad(source_pattern, 1, mode='edge')
for i in range(N):
    # i+1 pois o pad adicionou 1 no começo
    laplaciano_source[i] = pad_source[i] + pad_source[i+2] - 2*pad_source[i+1]

# 2. Montar o b final
b = laplaciano_source.copy()

# 3. Aplicar Condições de Contorno (Dirichlet)
# A primeira equação do sistema é: x_1 - 2x_0 + x_vizinho = laplaciano_0
# Mas x_vizinho (a esquerda de 0) é CONHECIDO (borda esquerda).
# Passamos ele para o lado do 'b' subtraindo.
b[0] -= altura_esquerda   # Subtrai pois passou pro outro lado da igualdade
b[-1] -= altura_direita   # Borda direita

# --- 3. A SOLUÇÃO ---
# Resolver o sistema linear Ax = b
# x é a nossa "Imagem Colada"
x = np.linalg.solve(A, b)

# --- 4. VISUALIZAÇÃO ---
print("\nResultado x (Os novos valores dos pixels):")
print(np.round(x, 1))

plt.figure(figsize=(10, 5))
plt.plot(source_pattern, 'r--', label='Original Source (Montanha no chão)')
plt.plot(x, 'b-o', label='Poisson Solution (Montanha elevada)')
plt.axhline(altura_esquerda, color='g', linestyle=':', label='Altura do Fundo (Target)')
plt.legend()
plt.title("Poisson Editing em 1D: A montanha subiu para encontrar a borda!")
plt.grid(True)
plt.show()